In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""),".."))

import custom
import numpy as np
import onnxruntime
import re


In [2]:

encoder = onnxruntime.InferenceSession("num_encoder.onnx", providers=["CPUExecutionProvider"])
decoder = onnxruntime.InferenceSession("num_decoder.onnx", providers=["CPUExecutionProvider"])


In [3]:
dic = {' ': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '0': 10, '+': 11}
query = None

while True :
    query = input("식을 입력하세요 (최대 길이 7) : ")
    if query == "종료" :
        break
    query = re.sub(r"[^0-9+]", "", string = query)
    query = list(query)
    query = custom.word_vectorize(query, dic, 7, False, " ", " ")
    query = [query]
    arr = np.array(query).astype(np.int64)
    
    inputs = {encoder.get_inputs()[0].name : arr}
    outputs = encoder.run(None, inputs)
    output = outputs[0]
    h = outputs[1]
    c = outputs[2]
    inputs = {decoder.get_inputs()[0].name : output, decoder.get_inputs()[1].name : h, decoder.get_inputs()[2].name : c}
    outputs = decoder.run(None, inputs)
    ys = outputs[0]
    
    for y in ys[0] :
        print(list(dic.keys())[y.argmax(axis = -1)], end = "")

식을 입력하세요 (최대 길이 7) :  31+47


77  

식을 입력하세요 (최대 길이 7) :  4+5


10  

식을 입력하세요 (최대 길이 7) :  8+7


...

식을 입력하세요 (최대 길이 7) :  3+3


7   

식을 입력하세요 (최대 길이 7) :  4+4


19  

식을 입력하세요 (최대 길이 7) :  종료
